<a href="https://colab.research.google.com/github/sainikhit2k/DL/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from keras.layers import Input, Flatten,Dense, Conv2D, MaxPool2D, ZeroPadding2D
from keras.models import Model


In [19]:
def conv_pool_block(x, n_filters, kernel_size, stride, padd=None, pool=True):

  if padd:
    x = ZeroPadding2D(padding=(padd,padd))(x)
  x = Conv2D(n_filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
  if pool:
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

  return x


def AlexNet(input_shape=(227, 227, 3)):
  inp_img = Input(shape=input_shape)
  out = conv_pool_block(inp_img, 96, (11,11), (4,4))
  out = conv_pool_block(out, 256, (5,5), (1,1), padd=2)
  out = conv_pool_block(out, 384, (3,3),(1,1), padd=1, pool=False)
  out = conv_pool_block(out, 384, (3,3),(1,1), padd=1, pool=False)
  out = conv_pool_block(out, 256, (3,3), (1,1), padd=1, pool=False)
  out = MaxPool2D(pool_size=(3,3), strides=(2, 2))(out)
  out = Flatten()(out)
  out = Dense(4096, activation='relu')(out)
  out = Dense(4096, activation='relu')(out)
  out = Dense(1000, activation='softmax')(out)
  model = Model(inp_img, out)
  model.summary()



alexNet = AlexNet()



Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_10 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 15, 15, 256)       0  